In [1]:
import pandas as pd
import numpy as np

### Import Data

In [2]:
restaurants_visitors_path = "Data set/restaurants_visitors.csv"
date_info_path = "Data set/date_info.csv"
store_info_path = "Data set/store_info.csv"

In [3]:
missing_values = ["#VALUE!"] #After a quick scan from the data, i found missing values where represented as "#VALUE!". I´ll consider #VALUE! as Nan.

restaurants_visitors_df = pd.read_csv(restaurants_visitors_path, na_values = missing_values, parse_dates=[2,3], dayfirst=True)
date_info_df = pd.read_csv(date_info_path, na_values = missing_values, parse_dates=[0])
store_info_df = pd.read_csv(store_info_path, na_values = missing_values) 

In [4]:
restaurants_visitors_df.head()

,id,visit_date,visit_datetime,reserve_datetime,reserve_visitors
0,db80363d35f10926,2016-01-01,2016-01-01 00:00:00,2016-01-01 01:00:00,5
1,db80363d35f10926,2016-01-02,2016-01-02 01:00:00,2016-01-01 16:00:00,2
2,db80363d35f10926,2016-01-02,2016-01-02 01:00:00,2016-01-01 15:00:00,4
3,db80363d35f10926,2016-01-02,2016-01-02 18:00:00,2016-01-02 14:00:00,2
4,db80363d35f10926,2016-01-02,2016-01-02 18:00:00,2016-01-02 02:00:00,3


In [7]:
date_info_df.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [8]:
store_info_df.head()

,store_id,genre_name,area_name,latitude,longitude
0,0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
1,7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
2,fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
3,a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
4,83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


### Data info

#### restaurants_visitors_df

In [9]:
restaurants_visitors_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12162 entries, 0 to 12161
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                12162 non-null  object        
 1   visit_date        8276 non-null   object        
 2   visit_datetime    12162 non-null  datetime64[ns]
 3   reserve_datetime  12162 non-null  datetime64[ns]
 4   reserve_visitors  12162 non-null  int64         
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 475.2+ KB


In [10]:
print(restaurants_visitors_df.isnull().sum())

id                     0
visit_date          3886
visit_datetime         0
reserve_datetime       0
reserve_visitors       0
dtype: int64


visit_date column is the only one with missing values in this dataframe. Since visit date can be extracted from visit_datatime. I´ll fill missing values from visit_data.

In [11]:
# replacing visit_date missing values with visit_datetime date
restaurants_visitors_df.visit_date.fillna(restaurants_visitors_df.visit_datetime, inplace=True)

In [21]:
# converting visit_date to date
restaurants_visitors_df['visit_date'] = pd.to_datetime(restaurants_visitors_df['visit_date'])
restaurants_visitors_df['visit_date'] = restaurants_visitors_df['visit_date'].dt.date

In [14]:
restaurants_visitors_df.describe()

,reserve_visitors
count,12162.000000
mean,4.040701
std,4.141749
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,56.000000


In [23]:
restaurants_visitors_df.loc[1580,]

id                     0a74a5408a0b8642
visit_date                   2016-10-27
visit_datetime      2016-10-27 18:00:00
reserve_datetime    2016-10-27 15:00:00
reserve_visitors                      2
Name: 1580, dtype: object

#### date_info_df

In [25]:
date_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   calendar_date  517 non-null    datetime64[ns]
 1   day_of_week    517 non-null    object        
 2   holiday_flg    517 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 12.2+ KB


#### store_info_df

In [26]:
store_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   store_id    829 non-null    object 
 1   genre_name  829 non-null    object 
 2   area_name   829 non-null    object 
 3   latitude    829 non-null    float64
 4   longitude   829 non-null    float64
dtypes: float64(2), object(3)
memory usage: 32.5+ KB


## Inserting data into SQL

Previosuly created DIDI_BI database in localhost and 3 tables to insert data

In [27]:
import pyodbc

In [28]:
conn_str = (
    r'DRIVER={SQL Server Native Client 11.0};'
    r'SERVER=localhost\SQLEXPRESS;'
    r'DATABASE=didi_bi;'
    r'Trusted_Connection=yes;'
)
cnxnn = pyodbc.connect(conn_str)

cursor = cnxnn.cursor()

In [29]:
#insert restaurants_visitors_df
for index,row in restaurants_visitors_df.iterrows():
    cursor.execute('INSERT INTO dbo.restaurants_visitors([id],[visit_date],[visit_datetime],[reserve_datetime], [reserve_visitors]) values (?,?,?,?,?)', 
                    row['id'], 
                    row['visit_date'], 
                    row['visit_datetime'],
                    row['reserve_datetime'],
                    row['reserve_visitors'])
    cnxnn.commit()

In [30]:
#insert store_info
for index,row in date_info_df.iterrows():
    cursor.execute('INSERT INTO dbo.date_info([calendar_date],[day_of_week],[holiday_flg]) values (?,?,?)', 
                    row['calendar_date'], 
                    row['day_of_week'], 
                    row['holiday_flg'])
    cnxnn.commit()

In [31]:
#insert date_info
for index,row in store_info_df.iterrows():
    cursor.execute('INSERT INTO dbo.store_info([store_id],[genre_name],[area_name], [latitude], [longitude]) values (?,?,?,?,?)', 
                    row['store_id'], 
                    row['genre_name'], 
                    row['area_name'],
                    row['latitude'],
                    row['longitude'])
    cnxnn.commit()

In [32]:
cursor.close()
cnxnn.close()